In [1]:
import geopandas as gpd
import h3
from shapely.geometry import Point, Polygon, LineString
import folium

In [2]:
build = gpd.read_file("데이터/F_FAC_BUILDING_서울/F_FAC_BUILDING_11_202210.shp", encoding = 'euc-kr')

In [3]:
build.head(3)

,UFID,BLD_NM,DONG_NM,GRND_FLR,UGRND_FLR,PNU,ARCHAREA,TOTALAREA,PLATAREA,HEIGHT,...,GB_CD,VIOL_BD_YN,GEOIDN,BLDG_PNU,BLDG_PNU_Y,BLD_UNLICE,BD_MGT_SN,SGG_OID,COL_ADM_SE,geometry
0,1991201839054527769900000000,None,None,3.0,0.0,1111017500107040000,57.6,203.13,100.6,9.1,...,0,0,B00100000000T30Z9,None,None,None,1111017500107040000001651,32,11110,"POLYGON ((201844.048 452780.137, 201842.638 45..."
1,1967201343844527773800000000,None,None,3.0,0.0,1111017500100560024,0.0,114.98,0.0,0.0,...,0,0,B00100000000T311B,None,None,None,1111017500100560024002963,34,11110,"POLYGON ((201344.164 452773.285, 201340.604 45..."
2,1962201375194527742000000000,None,None,2.0,0.0,1111017500100560054,0.0,135.20,0.0,0.0,...,0,0,B00100000000T313D,None,None,None,1111017500100560054002866,36,11110,"POLYGON ((201373.004 452770.525, 201372.704 45..."


In [4]:
build_height = build[['GRND_FLR', 'HEIGHT', 'geometry']]
build_height.head(3)

,GRND_FLR,HEIGHT,geometry
0,3.0,9.1,"POLYGON ((201844.048 452780.137, 201842.638 45..."
1,3.0,0.0,"POLYGON ((201344.164 452773.285, 201340.604 45..."
2,2.0,0.0,"POLYGON ((201373.004 452770.525, 201372.704 45..."


In [5]:
# 건물을 Point로 변환하기 위하여 중심점 추출
build_height['geometry'].centroid

0         POINT (201839.038 452776.991)
1         POINT (201343.829 452777.379)
2         POINT (201375.177 452774.201)
3         POINT (201388.462 452771.013)
4         POINT (200468.269 452776.403)
                      ...              
697882    POINT (201821.763 451827.109)
697883    POINT (192517.135 452812.706)
697884    POINT (202329.717 459965.755)
697885    POINT (201343.914 458685.038)
697886    POINT (200626.751 461379.041)
Length: 697887, dtype: geometry

In [6]:
build_point = build_height.copy()
build_point.head(3)

,GRND_FLR,HEIGHT,geometry
0,3.0,9.1,"POLYGON ((201844.048 452780.137, 201842.638 45..."
1,3.0,0.0,"POLYGON ((201344.164 452773.285, 201340.604 45..."
2,2.0,0.0,"POLYGON ((201373.004 452770.525, 201372.704 45..."


In [7]:
build_point['geometry'] = build_point['geometry'].centroid
build_point.head(3)

,GRND_FLR,HEIGHT,geometry
0,3.0,9.1,POINT (201839.038 452776.991)
1,3.0,0.0,POINT (201343.829 452777.379)
2,2.0,0.0,POINT (201375.177 452774.201)


In [8]:
build_point = build_point.to_crs(epsg = 4326)

In [9]:
lng = []; lat = []

for i in range(len(build_point)) :
    a = list(build_point['geometry'][i].coords)
    lat.append(a[0][1])
    lng.append(a[0][0])

In [10]:
build_point['lat'] = lat; build_point['lng'] = lng

In [11]:
build_point.head(3)

,GRND_FLR,HEIGHT,geometry,lat,lng
0,3.0,9.1,POINT (127.02161 37.57729),37.577287,127.021607
1,3.0,0.0,POINT (127.01600 37.57729),37.577291,127.016001
2,2.0,0.0,POINT (127.01636 37.57726),37.577262,127.016356


In [12]:
h3_9 = []

for i in range(len(build_point)) :
    h3_9.append(h3.geo_to_h3(build_point['lat'][i], build_point['lng'][i], 9))

build_point['h3_9'] = h3_9

In [13]:
def to_polygon(l) :
    return Polygon(h3.h3_to_geo_boundary(l, geo_json = True))

In [14]:
build_h3 = gpd.GeoDataFrame()
build_h3['GRND_FLR'] = build_point['GRND_FLR']
build_h3['HEIGHT'] = build_point['HEIGHT']
build_h3['geometry'] = build_point['h3_9'].apply(to_polygon)

In [15]:
build_h3 = build_h3.set_crs(epsg = 4326)
build_h3.head(3)

,GRND_FLR,HEIGHT,geometry
0,3.0,9.1,"POLYGON ((127.02359 37.57793, 127.02284 37.579..."
1,3.0,0.0,"POLYGON ((127.01816 37.57575, 127.01741 37.577..."
2,2.0,0.0,"POLYGON ((127.01816 37.57575, 127.01741 37.577..."


In [16]:
build_point['geometry'][0].within(build_h3['geometry'][0])

True

In [17]:
build_join = gpd.sjoin(build_h3, build_point, how = "left", op = "intersects")
build_join.head(3)

c:\Users\rproject\anaconda3\envs\geo\lib\site-packages\IPython\core\interactiveshell.py:3318: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [20]:
a.head(3)

,index_left,GRND_FLR_left,HEIGHT_left,GRND_FLR_right,HEIGHT_right,geometry,lat,lng,h3_9
0,NaN,NaN,NaN,3.0,9.1,POINT (127.02161 37.57729),37.577287,127.021607,8930e1c3263ffff
1,NaN,NaN,NaN,3.0,0.0,POINT (127.01600 37.57729),37.577291,127.016001,8930e1c324fffff
2,NaN,NaN,NaN,2.0,0.0,POINT (127.01636 37.57726),37.577262,127.016356,8930e1c324fffff


In [ ]:
# 서울시청 주소
lat = 37.5662; lng = 126.9787
lev8 = h3.geo_to_h3(lat, lng, 8)

In [ ]:
seoul = [lat, lng]